refer to https://www.kaggle.com/lordozvlad/tps-nov-logistic-regression-with-pytorch

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')

In [ ]:
x_data = train.drop(['id', 'target'], axis=1)
y_data = train.target

x_test = test.drop('id', axis=1)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data)

In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
import torch
import torchvision
from torch import nn

In [ ]:
class Net(nn.Module):
    def __init__(self,input_size,output_size):
        super(Net,self).__init__()
        self.f1 = nn.Linear(input_size, 300)
        self.f2 = nn.Linear(300, output_size)

    def forward(self,x):
        x = self.f1(x)
        x = F.relu(x)
        x = self.f2(x)
        return  torch.sigmoid(x)


In [ ]:
batch_size = 500
batch_no = len(x_train) // batch_size


In [ ]:
def generate_batches(X, y, batch_size):
    assert len(X) == len(y)
    np.random.seed(42)
    X = np.array(X)
    y = np.array(y)
    perm = np.random.permutation(len(X))

    for i in range(len(X)//batch_size):
        if i + batch_size >= len(X):
            continue
        ind = perm[i*batch_size : (i+1)*batch_size]
        yield (X[ind], y[ind])


In [ ]:
input_dim = x_train.shape[1]
output_dim = 2
learning_rate = 0.0001
model = Net(input_dim,output_dim)
error = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_list = []
roc_list = []
iteration_number = 150

for iteration in range(iteration_number):
    batch_loss = 0
    batch_roc = 0
    size = 0

    for (x, y) in generate_batches(x_train, y_train, batch_size):
        inputs = Variable(torch.from_numpy(x)).float()
        labels = Variable(torch.from_numpy(y))
            
        optimizer.zero_grad() 
        results = model(inputs)
        loss = error(results, labels)

        batch_loss += loss.data
        
        loss.backward()
        optimizer.step()
        
        batch_roc += roc_auc_score(labels.detach().numpy(), results[:, 1].detach().numpy())
        size += 1
    
    loss_list.append(batch_loss/batch_no)
    roc_list.append(batch_roc/size)
    
    if (iteration % 50 == 0):
        print('Epoch {}: loss {}, ROC {}'.format(iteration, batch_loss / batch_no, batch_roc / size))



In [ ]:
plt.plot(range(iteration_number), loss_list)
plt.xlabel("Number of Iterations")
plt.ylabel("Loss")
plt.show()
plt.plot(range(iteration_number), roc_list)
plt.xlabel("Number of Iterations")
plt.ylabel("ROC")
plt.show()

In [ ]:
x_test = Variable(torch.FloatTensor(x_test.values), requires_grad=False) 
y_test = model(x_test)


In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')
submission.target = y_test[:, 1].detach().numpy()
submission.to_csv('submission.csv', index=False)